In [4]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00


In [6]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.8 MB/s eta 0:00:00


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers import pipeline
import accelerate
import torch
import re

In [8]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    low_cpu_mem_usage=True,
    device_map="cuda:0"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [9]:
aspects = ['food', 'service', 'ambience', 'price', 'anecdotes/miscellaneous']
sentiments = ['positive', 'neutral', 'negative']

In [10]:
review_text = "I can understand the prices if it served better food, like some Chinese restaurants in midtown/uptown area."

In [11]:
# Using the text streamer to stream output one token at a time
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

sample_text = "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening"
sample_answer = {'service':'positive','food':'positive','price':'positive'}

prompt = f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation. For instance Review: {sample_text} has Answer: {sample_answer}  "
prompt_template=f'''<s>[INST] {prompt} [/INST]</s>[INST]{review_text}[/INST]'''

In [12]:
generation_params = {
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_new_tokens": 512,
    "repetition_penalty": 1.1
}

In [13]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    **generation_params
)

pipe_output = pipe(prompt_template)[0]['generated_text']
pipe_output.rsplit('[/INST]', 1)[-1]
pattern = re.compile(r'\{.*?\}')
match = pattern.search(pipe_output.rsplit('[/INST]', 1)[-1])
print(eval(match.group(0)))



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'price': 'neutral', 'food': 'positive'}


## Testing on eval dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
from datasets import load_from_disk

In [15]:
# Loading test data
dataset = load_from_disk("/content/drive/MyDrive/ds/test_dataset.hf")
dataset[0]

{'text': 'bread top notch well',
 'category': ['food'],
 'polarity': ['positive'],
 'joint': ['food#positive'],
 'category_labels': [0, 1, 0, 0, 0],
 'polarity_labels': [0, 0, 1],
 'joint_labels': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'true_labels': {'ambience': None,
  'food': 'positive',
  'other': None,
  'price': None,
  'service': None}}

In [16]:
def mistral_awq(data):
    # Using the text streamer to stream output one token at a time
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    sample_text = "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening"
    sample_answer = {'service':'positive','food':'positive','price':'positive'}

    generation_params = {
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.95,
        "top_k": 40,
        "max_new_tokens": 512,
        "repetition_penalty": 1.1
    }

    prompts = []
    for review_text in data["text"]:
        prompt = f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. Output should only contain mentioned aspects and their respective sentiments as a dictionary. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation. For instance Review: {sample_text} has Answer: {sample_answer}  "
        prompt_template=f'''<s>[INST] {prompt} [/INST]</s>[INST]{review_text}[/INST]'''
        prompts.append(prompt_template)

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, **generation_params)
    all_output = pipe(prompts)

    outputs = []
    for pipe_output in all_output:
        pipe_output = pipe_output[0]['generated_text']

        pipe_output.rsplit('[/INST]', 1)[-1]
        pattern = re.compile(r'\{.*?\}')
        match = pattern.search(pipe_output.rsplit('[/INST]', 1)[-1])
        if match is not None:
            outputs.append(match.group(0))
        else:
            outputs.append({})

    return {"mistral_awq": outputs}

In [17]:
dataset_awq = dataset.map(
    lambda _rows: mistral_awq(_rows), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

AttributeError: 'NoneType' object has no attribute 'group'